In [1]:
%env CUDA_VISIBLE_DEVICES=MIG-768d9c1d-110f-52e2-b0a2-3252f78280f8

env: CUDA_VISIBLE_DEVICES=MIG-768d9c1d-110f-52e2-b0a2-3252f78280f8


In [2]:
import io
import torch
import torch.nn as nn
import PIL.Image
import einops
import matplotlib.pyplot as plt
import numpy as np
import datasets
import math
import random
from timm.optim import Mars
from types import SimpleNamespace
from IPython.display import HTML
from types import SimpleNamespace
from fastprogress import progress_bar, master_bar
from torchvision.transforms.v2 import CenterCrop, RandomCrop
from torchvision.transforms.v2.functional import pil_to_tensor, to_pil_image
from decord import VideoReader
from autocodec.codec import AutoCodecND, latent_to_pil, pil_to_latent

In [3]:
device = "cuda"
dataset = datasets.load_dataset("danjacobellis/davis").cast_column('video',datasets.Video()).with_format("torch")

In [4]:
checkpoint = torch.load('checkpoint.pth', map_location="cpu",weights_only=False)
config = checkpoint['config']
state_dict = checkpoint['state_dict']
model = AutoCodecND(
    dim=3,
    input_channels=config.input_channels,
    J = int(np.log2(config.F)),
    latent_dim=config.latent_dim,
    encoder_depth = config.encoder_depth,
    encoder_kernel_size = config.encoder_kernel_size,
    decoder_depth = config.decoder_depth,
    lightweight_encode = config.lightweight_encode,
    lightweight_decode = config.lightweight_decode,
).to(device)
model.load_state_dict(state_dict)
model.eval();

In [5]:
def pad3d(x,p=16):
    b, c, f, h, w = x.shape
    t = math.ceil(f / p) * p
    fp1 = (t - f) // 2
    fp2 = (t - f) - fp1
    t = math.ceil(h / p) * p
    hp1 = (t - h) // 2
    hp2 = (t - h) - hp1
    t = math.ceil(w / p) * p
    wp1 = (t - w) // 2
    wp2 = (t - w) - wp1
    return torch.nn.functional.pad(x, pad=(wp1, wp2, hp1, hp2, fp1, fp2), mode="reflect")

In [6]:
μ1 = []
μ2 = []
max_frames_per_chunk = 48

for i_video in range(dataset['train'].num_rows):
    sample = dataset['train'][i_video]
    video = sample['video']
    len_video = len(video)
    x1080 = video.get_batch(range(len_video))
    x1080 = einops.rearrange(x1080, 'f h w c -> c f h w')
    x = []
    for i_frame in range(x1080.shape[1]):
        frame = x1080[:, i_frame]
        x.append(pil_to_tensor(to_pil_image(frame).resize((1920, 1080))).unsqueeze(1))
    x = torch.cat(x, dim=1).unsqueeze(0)
    x = x / 127.5 - 1.0
    x = x.to(device)
    x = pad3d(x)
    
    with torch.no_grad():
        z = model.encode(x)  # Latent representation
        latent = model.quantize.compand(z).round()  # Quantized latent
    
    num_frames = x.shape[2]
    num_chunks = math.ceil(num_frames / max_frames_per_chunk)
    chunk_sizes = [max_frames_per_chunk] * (num_chunks - 1)
    last_chunk_size = num_frames - (num_chunks - 1) * max_frames_per_chunk
    if last_chunk_size > 0:
        chunk_sizes.append(last_chunk_size)
    latent_chunk_sizes = [size // 16 for size in chunk_sizes]
    x_hat_chunks = []
    latent_start = 0
    with torch.no_grad():
        for chunk_size, latent_chunk_size in zip(chunk_sizes, latent_chunk_sizes):
            latent_end = latent_start + latent_chunk_size
            latent_chunk = latent[:, :, latent_start:latent_end]
            x_hat_chunk = model.decode(latent_chunk)
            x_hat_chunks.append(x_hat_chunk)
            latent_start = latent_end
    x_hat = torch.cat(x_hat_chunks, dim=2).clamp(-1,1)
    x_orig_01 = x / 2 + 0.5
    x_hat_01 = x_hat / 2 + 0.5
    PSNR = []
    for i_frame in range(x_orig_01.shape[2]):
        mse = torch.nn.functional.mse_loss(x_orig_01[0, :, i_frame], x_hat_01[0, :, i_frame])
        PSNR.append(-10 * mse.log10().item())
    
    μ1.append(np.mean(PSNR))
    μ2.append(np.median(PSNR))
    
print(f"Mean PSNR: {np.mean(μ1)}")

KeyboardInterrupt: 

In [7]:
μ1

[16.49607600023349,
 15.834108650684357,
 15.240096244961023,
 14.185478277504444,
 15.19011427462101,
 16.753116697072983,
 14.065810479223728,
 17.121395364403725,
 16.42669850339492,
 16.270789369940758,
 12.613061666488647,
 15.314476415514946,
 14.312519257267317,
 15.90378549695015,
 14.555015290776888,
 13.018711470067501,
 13.074974585324526,
 16.012832852346556]